In [1]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import pandas as pd

def load_from_tsfile(file_path, return_y=True):
    X, y = load_from_tsfile_to_dataframe(file_path)
    X = pd.DataFrame({i: pd.Series(x) for i, x in enumerate(X.iloc[:, 0])})
    if return_y:
        return X, y
    else:
        return X


# Test the function
dname= 'SpokenArabicDigits'
file_path = f'./datasets/{dname}/{dname}_TRAIN.ts'
X, y = load_from_tsfile(file_path)
print(X.shape, y.shape)



C:\Users\ui572274\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


(1152, 200) (200,)


In [2]:
file_path = f'./datasets/{dname}/{dname}_TEST.ts'
X_test, y_test = load_from_tsfile(file_path)
print(X_test.shape, y_test.shape)

(1152, 180) (180,)


In [3]:
X_train  = X.transpose()

In [4]:
X_test = X_test.transpose()

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
y = label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)


In [6]:
num_classes = len(np.unique(y_test))

num_classes

2

In [7]:
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    param = {
        'objective': 'binary',  # or 'multiclass' for multi-class classification
        'metric': 'binary_logloss',  # or 'multi_logloss' for multi-class classification

        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    }

    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy



In [9]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



[I 2024-02-08 17:43:26,999] A new study created in memory with name: no-name-c68997f9-c8b2-4ce9-991b-c0a2a4458328
[I 2024-02-08 17:43:27,513] Trial 0 finished with value: 0.5 and parameters: {'num_leaves': 54, 'learning_rate': 1.3157065213985106e-05, 'n_estimators': 534}. Best is trial 0 with value: 0.5.
[I 2024-02-08 17:43:27,793] Trial 1 finished with value: 0.5 and parameters: {'num_leaves': 164, 'learning_rate': 0.00015296158230257645, 'n_estimators': 183}. Best is trial 0 with value: 0.5.
[I 2024-02-08 17:43:28,407] Trial 2 finished with value: 0.5 and parameters: {'num_leaves': 134, 'learning_rate': 0.00019220153394893278, 'n_estimators': 683}. Best is trial 0 with value: 0.5.
[I 2024-02-08 17:43:29,105] Trial 3 finished with value: 0.5 and parameters: {'num_leaves': 232, 'learning_rate': 1.1598897242196882e-08, 'n_estimators': 777}. Best is trial 0 with value: 0.5.
[I 2024-02-08 17:43:29,286] Trial 4 finished with value: 0.46111111111111114 and parameters: {'num_leaves': 249, 'l